<div class="alert alert-block alert-info"> 

<b> Description </b> 
<br>
This is the RMM from ACCESS-S calulcated by Andrew Marshal. The original files
		are found in the e-mail from Andrew King and Andrew Marshal. His format was
		quite weird and thus there is a lot of processing that needed to be done to get 		it into the same format as the observed RMM, in order to be used in 
		pre-existing functions.
        
 </div>

In [1]:

import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask.array
import cartopy.crs as ccrs
import pickle
import matplotlib.colors as colors
import datetime as dt
import pickle
from matplotlib.colors import BoundaryNorm
rb = plt.cm.RdBu
bm = plt.cm.Blues
best_blue = '#9bc2d5'
recherche_red = '#fbc4aa'
wondeful_white = '#f8f8f7'
import glob
import pdb

'''Jupyter Markdown Cheat Sheet'''
# https://medium.com/ibm-data-science-experience/markdown-for-jupyter-notebooks-cheatsheet-386c05aeebed
# https://www.ibm.com/support/knowledgecenter/en/SSGNPV_1.1.3/dsx/markd-jupyter.html

'Jupyter Markdown Cheat Sheet'

<div class="alert alert-block alert-info"> <font size = "+6", color = 'black'> <center>Reading In </center></font></div>

In [2]:
'''There is a different file for every single month. Thus 12 files in total'''

rmm_path = '/home/563/ab2313/MJO/RMM/RMM_access_raw/'
rmm_files = glob.glob(rmm_path + '*.nc')
rmm_files

['/home/563/ab2313/MJO/RMM/RMM_access_raw/RMM12_day01_month01_1990-2012_ncep_access-s1.nc',
 '/home/563/ab2313/MJO/RMM/RMM_access_raw/RMM12_day01_month02_1990-2012_ncep_access-s1.nc',
 '/home/563/ab2313/MJO/RMM/RMM_access_raw/RMM12_day01_month03_1990-2012_ncep_access-s1.nc',
 '/home/563/ab2313/MJO/RMM/RMM_access_raw/RMM12_day01_month04_1990-2012_ncep_access-s1.nc',
 '/home/563/ab2313/MJO/RMM/RMM_access_raw/RMM12_day01_month05_1990-2012_ncep_access-s1.nc',
 '/home/563/ab2313/MJO/RMM/RMM_access_raw/RMM12_day01_month06_1990-2012_ncep_access-s1.nc',
 '/home/563/ab2313/MJO/RMM/RMM_access_raw/RMM12_day01_month09_1990-2012_ncep_access-s1.nc',
 '/home/563/ab2313/MJO/RMM/RMM_access_raw/RMM12_day01_month10_1990-2012_ncep_access-s1.nc',
 '/home/563/ab2313/MJO/RMM/RMM_access_raw/RMM12_day01_month11_1990-2012_ncep_access-s1.nc',
 '/home/563/ab2313/MJO/RMM/RMM_access_raw/RMM12_day01_month12_1990-2012_ncep_access-s1.nc',
 '/home/563/ab2313/MJO/RMM/RMM_access_raw/RMM12_day01_month07_1990-2012_ncep_acc

<div class="alert alert-block alert-success">
<br>
<font size = "+6", color = 'black'> <center> Processing </center></font>
<br>

<font color = 'black'> <center> Changing the format so that it is useable </center></font>

</div>

In [6]:
'''
1. Looping throught all the files
1.5 Finding the month in this file
2. There are 11 differetn ensemble members, each one needs to be done individually.
3. Each point is referenced by year, which marks the start of the run, and then by the days since this. 
   need to convert this into a plain time series
4. The format that I want is datetime
5. Gettging the actual dates that things occured at. Oringonal file has a start date, and then days since then.
   I'm putting this as just the actual date this occured at. 
6. Storing everything as an xarray file
7. For each file we only want a single month of the data
8. Cominging into the one xarry file. Prior to this all of the RMM's are just for a single month
'''



total_stor = []

'''1. File loop'''
for file in rmm_files:
    
    '''1.5 Working out what month we are dealing with in this file. Usefule for later on'''
    month = list(file.split('/')[-1].split('_')[2])[-2:]
    month = month[0] + month[1]
    month = int(month)

    
    single_file = xr.open_dataset(file) # Opening a file
    single_file.attrs = {} # Remving all the attribues
    

    # Storage for all ensembles
    time_ensemble = []
    rmm1_ensemble = []
    rmm2_ensemble = []
    
    
    '''2. Ensemlbe loop'''
    ensemble_members = single_file.nvars.values
    for ensemble in ensemble_members:

        single_ensemble =single_file.sel(nvars = ensemble).drop('nvars') # Selecting a single ensemble

        
        time_series = []
        rmm1_series = []
        rmm2_series = []
        
        '''3. Getting the start'''
        init_years = single_ensemble.nfc.values
        for init in init_years:

            '''4. Converting everything into a datetime format. The format is YYYYMMDD'''
            year = int(np.trunc(init/10000)) #By dividing by 10000, and then truncating this will just give the year
            month = int(np.trunc(init/100)) - year * 100 # Same logic, but now must remove the year
            day = int(init - month * 100 - year * 10000) # Same logica as line above
            # Converting this into a dattime format
            init_date = pd.to_datetime(str(year) +'-'+ str(month) +'-'+ str(day), format = '%Y-%m-%d')
            

            # All these values are in nanoseconds (can't explain why, very strange), thus they need to be converted
            nanosecond_to_day_conversion =  10 **-9/60/60/24 
            raw_time_deltas = single_ensemble.time.values
            # Converting from a time_delta in nanoseconds to days. Time deltas looks like [0,1,2,3,4]
            time_deltas = (np.array(raw_time_deltas) * nanosecond_to_day_conversion).astype(int) - 1


            '''5. Actual Dates'''
            # Looping throught the time_delatas, and the raw. The raw is there so I can extract
            # the RMM1 and RMM2 values form the intial array
            for delta, raw in zip(time_deltas, raw_time_deltas):
                actual_date = init_date + pd.to_timedelta(str(delta) + 'D') #Shifting initial date forward
                time_series.append(actual_date)
                
                # Extracting the RMM
                RMM_val = single_ensemble.RMM12.sel(nfc = init, time = raw).values 
                rmm1_series.append(RMM_val[0])
                rmm2_series.append(RMM_val[1])

        time_ensemble.append(time_series)   
    #     time_ensemble = np.array(time_ensemble)
        rmm1_ensemble.append(rmm1_series)
        rmm2_ensemble.append(rmm2_series)

    '''6. Making into an xarray file'''
    rmm_access_single_month = xr.Dataset(
    {'rmm1':(('ensemble','time'), rmm1_ensemble), 'rmm2':(('ensemble','time'), rmm2_ensemble)},
    {'ensemble': np.arange(12),
     'time': time_ensemble[0]
    }
    )
    
    '''7. Single Month Select'''
    
    rmm_access_single_month = rmm_access_single_month.where(rmm_access_single_month.time.dt.month == month, 
                                                            drop = True)
    
    #########################
    total_stor.append(rmm_access_single_month)
     

'''8. Combinging into a single file'''   
for i,file in enumerate(total_stor):
    if i == 0:
        rmm_access_raw = file

    else:
        rmm_access_raw = rmm_access_raw.combine_first(file)
            

In [19]:
rmm_access_raw

<xarray.Dataset>
Dimensions:   (ensemble: 12, time: 8401)
Coordinates:
  * time      (time) datetime64[ns] 1990-01-01 1990-01-02 ... 2012-12-31
  * ensemble  (ensemble) int64 0 1 2 3 4 5 6 7 8 9 10 11
Data variables:
    rmm1      (ensemble, time) float32 0.058957588 0.15294036 ... 1.5933019
    rmm2      (ensemble, time) float32 -1.9142343 -1.9791112 ... -0.6532849

<div class="alert alert-block alert-info"> <font color = 'black'>Saving the file and creating a text file with information </font></div>

In [8]:
rmm_access_raw.to_netcdf('rmm_access_raw.nc')

In [9]:
%%writefile rmm_access_raw.txt
The file rmm_access_raw.nc contains the RMM from Andrew Marshal. This file contains just the raw values
of RMM1 and RMM2. They have not yet been converted into a phase of an amplitude

Overwriting rmm_access_raw.txt
